In [1]:
# Step 1.a: import all needed modules from Python/NRPy+:
import sympy as sp                # SymPy: The Python computer algebra package upon which NRPy+ depends
import sys, os     # Standard Python modules for multiplatform OS-level functions
sys.path.append('../')

from outputC import superfast_uniq, lhrh      # Remove duplicate entries from a Python array; store left- and right-
                                              #   hand sides of mathematical expressions

# As of April 2021, "sp.sympify("Q+1")" fails because Q is a reserved keyword.
#   This is the workaround, courtesy Ken Sible.
custom_global_dict = {}
exec('from sympy import *', custom_global_dict)
del custom_global_dict['Q']

# Step 1.b: Check for a sufficiently new version of SymPy (for validation)
# Ignore the rc's and b's for release candidates & betas.
sympy_version = sp.__version__.replace('rc', '...').replace('b', '...')
sympy_version_decimal = float(int(sympy_version.split(".")[0]) + int(sympy_version.split(".")[1])/10.0)
if sympy_version_decimal > 1.2:
    custom_parse_expr = lambda expr: sp.parse_expr(expr, global_dict=custom_global_dict)
else:
    custom_parse_expr = lambda expr: sp.sympify(expr)

if sympy_version_decimal < 1.2:
    print('Error: NRPy+ does not support SymPy < 1.2')
    sys.exit(1)

# Step 1.c: Name of the directory containing the input file
inputdir = "Hamiltonian"

In [2]:
# Step 2.a: Read in expressions as a (single) string
with open(os.path.join(inputdir,'Sympy_Hreal_on_Bottom.txt'), 'r') as file:
    expressions_as_lines = file.readlines()

#print(expressions_as_lines)
# Step 2.b: Create and populate the "lr" array, which separates each line into left- and right-hand sides
#   Each entry is a string of the form lhrh(lhs='',rhs='')
lr = []

for i in range(len(expressions_as_lines)):
    # Ignore lines with 2 or fewer characters and those starting with #
    if len(expressions_as_lines[i]) > 2 and expressions_as_lines[i][0] != "#":
        # Split each line by its equals sign
        split_line = expressions_as_lines[i].split("=")
        #print(split_line)
        # Append the line to "lr", removing spaces, "sp." prefixes, and replacing Lambda->Lamb
        #   (Lambda is a protected keyword):
        lr.append(lhrh(lhs=split_line[0].replace(" ","").replace("Lambda","Lamb"),
                       rhs=split_line[1].replace(" ","").replace("sp.","").replace("Lambda","Lamb")))

# Step 2.c: Separate and sympify right- and left-hand sides into separate arrays
lhss = []
rhss = []
for i in range(len(lr)):
    #print(lr[i].rhs)
    lhss.append(custom_parse_expr(lr[i].lhs))
    rhss.append(custom_parse_expr(lr[i].rhs))

In [3]:
# Step 3.a: Create `input_constants` array and populate with SymPy symbols
m1,m2,tortoise,eta,EMgamma = sp.symbols('m1 m2 tortoise eta EMgamma',
                                                                real=True)
input_constants = [m1,m2,tortoise,eta,EMgamma]

# Step 3.b: Create `dynamic_variables` array and populate with SymPy symbols
#x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z = sp.symbols("x y z p1 p2 p3 S1x S1y S1z S2x S2y S2z", real=True)
x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z = sp.symbols("x y z p1 p2 p3 S1x S1y S1z S2x S2y S2z")
dynamic_variables = [x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z]

In [4]:
# Step 4.a: Prepare array of "free symbols" in the right-hand side expressions
full_symbol_list_with_dups = []
for i in range(len(lr)):
    for variable in rhss[i].free_symbols:
        full_symbol_list_with_dups.append(variable)

# Step 4.b: Remove duplicate free symbols
full_symbol_list = superfast_uniq(full_symbol_list_with_dups)

# Step 4.c: Remove input constants from symbol list
for inputconst in input_constants:
    for symbol in full_symbol_list:
        if str(symbol) == str(inputconst):
            full_symbol_list.remove(symbol)
for dynamic_variable in dynamic_variables:
    for symbol in full_symbol_list:
        if str(symbol) == str(dynamic_variable):
            full_symbol_list.remove(symbol)


In [5]:
# Step 5.b: Mark each free variable as a function with argument xx
full_function_list = []
for symb in full_symbol_list:
    func = sp.sympify(sp.Function(str(symb),real=True)(x,y,z,p1,p2,p3,S1x,S1y,S1z,S2x,S2y,S2z))
    full_function_list.append(func)
    for i in range(len(rhss)):
        for var in rhss[i].free_symbols:
            if str(var) == str(symb):
                rhss[i] = rhss[i].subs(var,func)

x/sqrt(x**2 + y**2 + z**2)


In [6]:
# Step 6: Use SymPy's diff function to differentiate right-hand sides with respect to xx
#   and append "prm" notation to left-hand sides
lhss_deriv = []
rhss_deriv = []
for i in range(len(rhss)):
    for dynamic_variable in dynamic_variables:
        var = str(dynamic_variable)
        lhss_deriv.append(custom_parse_expr(str(lhss[i])+"prm"+var))
        newrhs = custom_parse_expr(str(sp.diff(rhss[i],dynamic_variable)).replace("(x, y, z, p1, p2, p3, S1x, S1y, S1z, S2x, S2y, S2z)","").replace(", "+var+")","prm"+var).replace("Derivative(",""))
        rhss_deriv.append(newrhs)

In [7]:
# Step 7.a: Define derivative simplification function
def simplify_deriv(lhss_deriv,rhss_deriv):
    # Copy expressions into another array
    lhss_deriv_simp = []
    rhss_deriv_simp = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_simp.append(lhss_deriv[i])
        rhss_deriv_simp.append(rhss_deriv[i])
    # If a right-hand side is 0, substitute value 0 for the corresponding left-hand side in later terms
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == 0:
            for j in range(i+1,len(rhss_deriv_simp)):
                for var in rhss_deriv_simp[j].free_symbols:
                    if str(var) == str(lhss_deriv_simp[i]):
                        rhss_deriv_simp[j] = rhss_deriv_simp[j].subs(var,0)
    zero_elements_to_remove = []
    # Create array of indices for expressions that are zero
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == sp.sympify(0):
            zero_elements_to_remove.append(i)
    # When removing terms that are zero, we need to take into account their new index (after each removal)
    count = 0
    for i in range(len(zero_elements_to_remove)):
        del lhss_deriv_simp[zero_elements_to_remove[i]+count]
        del rhss_deriv_simp[zero_elements_to_remove[i]+count]
        count -= 1
    return lhss_deriv_simp,rhss_deriv_simp

# Step 7.b: Call the simplication function and then copy results
lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv,rhss_deriv)
lhss_deriv = lhss_deriv_simp
rhss_deriv = rhss_deriv_simp

In [8]:
# Step 9: Output original expression and each partial derivative expression in SymPy snytax
outputdir = "Derivatives"
with open(os.path.join(outputdir,'partial_derivatives.txt-VALIDATION'), 'w') as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                "sp.Abs(").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    for i in range(len(lhss_deriv)):
            right_side = str(rhss_deriv[i])
            right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                "sp.Abs(").replace("Rational(","sp.Rational(").replace("prm",
                                                "_prm")
            output.write(str(lhss_deriv[i]).replace("prm","_prm")+" = "+right_side_in_sp+"\n")

In [9]:
# Step 10: Create Numpy function to output exact derivatives for all 12 dynamical variables

with open(os.path.join(outputdir,'hamiltonian_first_derivatives.py'),"w") as file:
    file.write("""from __future__ import division
import numpy as np
def ham_first_derivs(m1, m2, tortoise, x, y, z, p1, p2, p3, S1x, S1y, S1z, S2x, S2y, S2z):
""")
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_np = right_side.replace("sqrt(","np.sqrt(").replace("log(","np.log(").replace("pi",
                                                "np.pi").replace("sign(","np.sign(").replace("Abs(",
                                                "np.abs(").replace("Rational(","np.true_divide(")
        file.write("    " + lr[i].lhs + " = " + right_side_in_np)
    for i in range(len(lhss_deriv)):
        right_side = str(rhss_deriv[i])
        right_side_in_np = right_side.replace("sqrt(","np.sqrt(").replace("log(","np.log(").replace("pi",
                                                "np.pi").replace("sign(","np.sign(").replace("Abs(",
                                                "np.abs(").replace("Rational(","np.true_divide(").replace("prm",
                                                "_prm")
        file.write("    " + str(lhss_deriv[i]).replace("prm", "_prm") + " = " + right_side_in_np + "\n")
    file.write("    return np.array([Hreal_prmx, Hreal_prmy, Hreal_prmz, Hreal_prmp1, Hreal_prmp2, Hreal_prmp3, Hreal_prmS1x, Hreal_prmS1y, Hreal_prmS1z, Hreal_prmS2x, Hreal_prmS2y, Hreal_prmS2z])\n")
    

In [14]:
## Validate against LALSuite
import numpy as np
from Derivatives.hamiltonian_first_derivatives import ham_first_derivs as dH

for index in range(3):
    validationfile = 'outputv4Pindex'+str(index)+'.txt'
    validationfile_pert = 'outputv4Pindex'+str(index)+'pert.txt'
    with open(os.path.join(outputdir,validationfile)) as datfile, open(os.path.join(outputdir,validationfile_pert)) as pertdatfile:
        data_lines = datfile.readlines()
        pertdata_lines = pertdatfile.readlines()
        for i in range(len(pertdata_lines)//3):
            valuepairs = data_lines[3*i].strip('{}\n').split(',')
            valuepairspert = pertdata_lines[3*i].strip('{}\n').split(',')
            derivpairs = data_lines[3*i+1].strip('{}\n').split(',')
            derivpairspert = pertdata_lines[3*i+1].strip('{}\n').split(',')
            values = dict()
            derivs = dict()
            valuespert = dict()
            derivspert = dict()
            for j in range(len(valuepairs)):
                pair = valuepairs[j].split(': ')
                pairpert = valuepairspert[j].split(': ')
                values[pair[0].strip(" ''")] = float(pair[1])
                valuespert[pairpert[0].strip(" ''")] = float(pairpert[1])
            for j in range(len(derivpairs)):
                pair = derivpairs[j].split(': ')
                pairpert = derivpairspert[j].split(': ')
                derivs[pair[0].strip(" ''")] = float(pair[1])
                derivspert[pairpert[0].strip(" ''")] = float(pairpert[1])
            
            m1 , m2 = values['m1'], values['m2']
            eta = m1*m2/(m1 + m2)/(m1 + m2)
            
            nrpy_derivs = dH(values['m1'],values['m2'],values['tortoise'],values['x'],values['y'],values['z'],values['p1'],values['p2'],values['p3'],values['S1x'],values['S1y'],values['S1z'],values['S2x'],values['S2y'],values['S2z'])
            for j in range(len(dynamic_variables)):
                var = str(dynamic_variables[j])
                Erel = np.abs((nrpy_derivs[j]/eta - derivs['dHreal_d'+var])/derivs['dHreal_d'+var])
                Erelpert = np.abs((derivs['dHreal_d'+var]-derivspert['dHreal_d'+var])/derivs['dHreal_d'+var])
                if Erel > Erelpert:
                    print("In line %d: Erel in %s derivative is %.2e, allowed is %.2e"%(i,var,Erel,Erelpert))
            
            
        
    


In line 0: Erel in y derivative is 1.33e-03, allowed is 2.11e-05
In line 0: Erel in z derivative is 1.53e+03, allowed is 1.01e-07
In line 0: Erel in p1 derivative is 1.85e-01, allowed is 0.00e+00
In line 0: Erel in p3 derivative is 6.26e-09, allowed is 2.79e-09
In line 0: Erel in S1y derivative is 1.19e-06, allowed is 4.73e-08
In line 0: Erel in S2x derivative is 1.26e-07, allowed is 6.32e-08
In line 0: Erel in S2y derivative is 1.77e-06, allowed is 1.57e-06
In line 1: Erel in x derivative is 3.16e-10, allowed is 1.56e-10
In line 1: Erel in y derivative is 1.46e-03, allowed is 1.48e-04
In line 1: Erel in z derivative is 1.53e+03, allowed is 7.09e-07
In line 1: Erel in p1 derivative is 1.85e-01, allowed is 1.22e-04
In line 1: Erel in p2 derivative is 2.47e-11, allowed is 2.57e-12
In line 1: Erel in S1y derivative is 1.20e-06, allowed is 1.58e-07
In line 2: Erel in x derivative is 2.40e-10, allowed is 4.25e-11
In line 2: Erel in y derivative is 1.33e-03, allowed is 2.11e-05
In line 2: Er

<ipython-input-14-a2c893d53833>:37: RuntimeWarning: divide by zero encountered in double_scalars
  Erel = np.abs((nrpy_derivs[j]/eta - derivs['dHreal_d'+var])/derivs['dHreal_d'+var])


ZeroDivisionError: float division by zero